In [1]:
# Load necessary libs and set seed
set.seed(1234)

library(tidyverse)
library(tidymodels)
library(repr)
library(gridExtra)
library(faraway)
library(mltools)
library(leaps)
library(glmnet)
library(cowplot)
library(MASS)
library(boot)
library(caret)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

# Introduction

We plan on using the 'Adult' dataset (https://archive.ics.uci.edu/dataset/2/adult) from the UC Irvine Machine Learning Repository. The 'Adult' dataset was extracted from the 1994 Census Bureau database and compiled by Ronny Kohavi and Barry Becker. The dataset contains general identification information on the average adult about their personal and work lives. Our adult dataset consists of 48,842 instances or observations. If records with unknown values are removed, it contains 45,222 observations. With an abundance of data to work with, our primary goal is to target the 'income’ variable which is a binary attribute that classifies individuals into two categories: those who earn less than or equal to 50000 dollars (<=50K) and those who earn more than $50,000 (>50K) and be able to make accurate predictions about it. To begin, allow us to describe the variables we’ll be working with in the following overview about the variable:

| Feature         | Description |
|:----------------|:------------|
| `age`           | A continuous feature representing the age of the individual in years. |
| `workclass`     | A categorical feature categorizing individuals into employment categories such as Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. This variable contains missing values. |
| `fnlwgt`        | A continuous attribute that estimates the number of people in the population with the same demographics as the individual. |
| `education`     | A categorical feature indicating the highest level of education completed by the individual, with options like Bachelors, Some-college, 11th, HS-grad, and more. |
| `education_num` | A numeric representation (continuous) of the highest education level completed by the individual, ranging from 1 to 16. |
| `marital_status`| A categorical feature denoting the marital status of the individual, with options like Married-civ-spouse, Divorced, Never-married, and more. |
| `occupation`    | A categorical feature representing the job of the individual. Occupation categories include Tech-support, Craft-repair, and others. This variable also contains missing values. |
| `relationship`  | A categorical feature describing the individual's relationship status, with categories like Wife, Own-child, Husband, and more. |
| `race`          | A categorical attribute indicating the individual's race, with options like White, Asian-Pac-Islander, and others. |
| `sex`           | A binary attribute classifying the individual's gender as either male or female. |
| `capital_gain`  | A continuous attribute representing the capital gain in the previous year. |
| `capital_loss`  | A continuous attribute representing the capital loss in the previous year. |
| `hours_per_week`| A continuous attribute indicating the number of hours worked per week by the individual. |
| `native_country`| A categorical feature specifying the native country of the individual, with categories including the United-States, Cambodia, England, and more. This variable also contains missing values. |
| `income`        | A binary attribute that encodes whether the individual earns <=50K or >50K dollars. |


Generally speaking, we want to answer the following question: "which condition factors, when used as explanatory variables, create the model that yields the most accurate real-time predictions of annual income in North America?" We intend to utilize the dataset to develop a predictive model that can forecast an individual's income level (the binary response variable) by taking necessary factors into account. The goal is to use the available data to create a predictive tool that can estimate whether an individual earns less than or equal to 50,000 dollars or more than $50,000 annually. Given that the response variable 'income' is binary, logistic regression appears to be a suitable technique for this prediction task. Logistic regression is a powerful technique which gives us the ability to determine the probability of a binary outcome (in this case, earning more or less/equal to 50k) given other input variables. The dataset contains a wealth of information on these variables, which will be instrumental in constructing and training an accurate logistic regression model. 


The distribution of income has been a significant focus in economic and social research. Studies have explored various factors influencing an individual's earning potential, contributing valuable insights into societal trends and disparities. The paper by Harmon, Oosterbeek, and Walker (2003) titled "The Returns to Education: Microeconomics"  is a pivotal contribution to understanding the relationship between education and income levels. This paper extensively investigates the economic aspects of education by focusing on the returns individuals receive from investing in education.
By analyzing the relationships and patterns within the variables in our dataset, we aim to make informed predictions about an individual's income level. Furthermore, Hegewisch and Hartmann’s paper on gender wage gap sheds light on the persistence of occupational gender segregation in the labor market. It highlights the stagnation in integrating occupations across various demographics, including different generations, educational levels, and racial and ethnic groups, particularly over the past decade. There are many other researches that discuss the influence of various factors on income. We want to examine what combination of factors are the most influential in terms of determining an individual’s earning potential. It can have applications in diverse fields, such as human resources, financial planning, and public policy.

To begin, we will first read our dataset into R and perform general cleaning and wrangling to make the analysis easier.


# Methods and Results

## Cleaning and Wrangling Data

In order to enhance the quality and clarity of our data analysis, we will begin by assigning meaningful column names to our variables. This practice not only helps us maintain data integrity but also makes the subsequent analysis more accessible and interpretable.Our analysis is primarily centered around choosing variables that are the most statistically significant in predicting income levels.Furthermore, to ensure the reliability of our analysis, we will filter out any records with missing values in the 'income,' ‘workclass,’ and ‘occupation’ column. This step ensures that our results are based on complete and accurate data. Regarding the choice between 'education_num' and 'education,' we opt for 'education_num' due to its numerical representation of education levels. This numeric format is advantageous for statistical modeling, as it can capture ordinal relationships and lead to more straightforward analyses. In contrast, 'education' exhibits a significantly higher cardinality, which can result in longer processing times and potentially complicate the analysis. Furthermore, we also dropped the country column because it is a redundant variable in describing ethnicity and we already use race to do this. It's important to note that the exclusion of other variables is deliberate. We hope to gain insights into what variables have the most statistically significant impact in determining the income level. 

In [2]:
#Read and tidy data
adult_test <- read.table("https://github.com/Haobo11/STAT301-Group-Project/raw/main/adult.test",  sep = "," , skip=1)
colnames(adult_test) <- c("age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
                          "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss",
                          "hours_per_week", "native_country", "income")
adult_test <- adult_test|>
    dplyr::select(- capital_gain, - capital_loss, - education_num, - workclass)|> #Remove the capital_gain and capital_loss column, 
                                                 #since there are many missing value in thest two variable.
    mutate(                              #Factor the categorical variable
           education = factor(education),
           marital_status = factor(marital_status),
           occupation = factor(occupation),
           relationship = factor(relationship),
           race = factor(race),
           sex = factor(sex),
           native_country = factor(native_country),
           income = factor(income))|>
    mutate(across(where(is.character), str_trim))|>
    filter(occupation!=" ?")|>
    filter(native_country!=" ?")|> #Filter the observation that have missing value
    filter(native_country == c(" United-States", " Canada", " Mexico", " Jamaica", " Cuba"))#Since our quetion focused on the data in North America, 
                                                                                             # we filter the North America Country in Dataset.
head(adult_test)

adult_data <- read.table("https://github.com/Haobo11/STAT301-Group-Project/raw/main/adult.data",  sep = ",")
colnames(adult_data) <- c("age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
                          "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss",
                          "hours_per_week", "native_country", "income")
adult_data <- adult_data|>
     dplyr::select(- capital_gain, - capital_loss,  - education_num, - workclass)|> #Remove the capital_gain and capital_loss column, 
                                                 #since there are many missing value in thest two variable.
    mutate(#Factor the categorical variable
           education = factor(education),
           marital_status = factor(marital_status),
           occupation = factor(occupation),
           relationship = factor(relationship),
           race = factor(race),
           sex = factor(sex),
           native_country = factor(native_country),
           income = factor(income))|>
    filter(occupation!=" ?") |>
    filter(native_country!=" ?")|> #Filter the observation that have missing value
    filter(native_country == c(" United-States", " Canada", " Mexico", " Jamaica", " Cuba")) #Since our quetion focused on the data in North America, 
                                                                                             # we filter the North America Country in Dataset.
head(adult_data)

,age,fnlwgt,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income
,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>
1,25,226802,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,40,United-States,<=50K.
2,63,104626,Prof-school,Married-civ-spouse,Prof-specialty,Husband,White,Male,32,United-States,>50K.
3,26,82091,HS-grad,Never-married,Adm-clerical,Not-in-family,White,Female,39,United-States,<=50K.
4,37,60548,HS-grad,Widowed,Machine-op-inspct,Unmarried,White,Female,20,United-States,<=50K.
5,45,432824,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,90,United-States,>50K.
6,46,106444,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,38,United-States,>50K.


Warning message:
“There were 2 warnings in `filter()`.
The first warning was:
ℹ In argument: `==...`.
Caused by warning in `==.default`:
! longer object length is not a multiple of shorter object length
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning.”


,age,fnlwgt,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income
,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>
1,39,77516,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K
2,28,338409,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K
3,37,284582,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,United-States,<=50K
4,37,280464,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80,United-States,>50K
5,25,176756,HS-grad,Never-married,Farming-fishing,Own-child,White,Male,35,United-States,<=50K
6,54,302146,HS-grad,Separated,Other-service,Unmarried,Black,Female,20,United-States,<=50K


In [3]:
# Assuming adult_data is our training dataset and adult_test is our testing dataset
adult_data <- 
    adult_data %>% 
    mutate(income = ifelse(income == " >50K", 1, 0))
head(adult_data)

adult_test <- 
    adult_test %>% 
    mutate(income = ifelse(income == " >50K.", 1, 0))
head(adult_test)
# Fitting the logistic regression model with stepwise selection using BIC
full_model <- glm(income ~ ., data = adult_data, family = binomial())
full_model_result<-tidy(full_model)
full_model_result
stepwise_model <- stepAIC(full_model, direction = "both", trace = FALSE, k = log(nrow(adult_data)))

# Summarizing the model
summary(stepwise_model)

# Predicting and evaluating on the test set
predictions <- round(predict(stepwise_model, newdata = adult_test, type = "response"),0)

# Use the confusionMatrix function from the caret package
adult_income_confusion_matrix <- 
    confusionMatrix(
    data = as.factor(predictions),
    reference = as.factor(adult_test$income),
    positive = "1"
)
adult_income_confusion_matrix

,age,fnlwgt,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income
,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<dbl>
1,39,77516,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,0
2,28,338409,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,0
3,37,284582,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,United-States,0
4,37,280464,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80,United-States,1
5,25,176756,HS-grad,Never-married,Farming-fishing,Own-child,White,Male,35,United-States,0
6,54,302146,HS-grad,Separated,Other-service,Unmarried,Black,Female,20,United-States,0


,age,fnlwgt,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income
,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<dbl>
1,25,226802,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,40,United-States,0
2,63,104626,Prof-school,Married-civ-spouse,Prof-specialty,Husband,White,Male,32,United-States,1
3,26,82091,HS-grad,Never-married,Adm-clerical,Not-in-family,White,Female,39,United-States,0
4,37,60548,HS-grad,Widowed,Machine-op-inspct,Unmarried,White,Female,20,United-States,0
5,45,432824,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,90,United-States,1
6,46,106444,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,38,United-States,1


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-8.009135e+00,1.129993e+00,-7.08777243,1.362878e-12
age,3.452366e-02,3.657134e-03,9.44008492,3.724774e-21
fnlwgt,7.610529e-07,3.788911e-07,2.00863221,4.457615e-02
education 11th,-3.284171e-01,4.991910e-01,-0.65789870,5.106032e-01
education 12th,2.791073e-01,6.744894e-01,0.41380529,6.790167e-01
education 1st-4th,-5.213018e-01,1.135659e+00,-0.45903012,6.462125e-01
education 5th-6th,-1.363546e+01,3.071465e+02,-0.04439399,9.645904e-01
education 7th-8th,-5.741656e-01,5.081154e-01,-1.12999061,2.584802e-01
education 9th,2.051449e-01,5.643542e-01,0.36350374,7.162286e-01


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


Call:
glm(formula = income ~ age + education + occupation + relationship + 
    sex + hours_per_week, family = binomial(), data = adult_data)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.61237  -0.58563  -0.21380  -0.00005   3.11921  

Coefficients:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -5.701e+00  4.776e-01 -11.936  < 2e-16 ***
age                           3.610e-02  3.506e-03  10.298  < 2e-16 ***
education 11th               -3.385e-01  4.988e-01  -0.679  0.49733    
education 12th                2.880e-01  6.723e-01   0.428  0.66835    
education 1st-4th            -7.738e-01  1.107e+00  -0.699  0.48441    
education 5th-6th            -1.368e+01  3.131e+02  -0.044  0.96514    
education 7th-8th            -6.195e-01  5.080e-01  -1.220  0.22260    
education 9th                 2.214e-01  5.584e-01   0.397  0.69171    
education Assoc-acdm          1.050e+00  4.115e-01   2.551  0.01074 * 

Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1931  317
         1  177  408
                                          
               Accuracy : 0.8256          
                 95% CI : (0.8111, 0.8394)
    No Information Rate : 0.7441          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.5112          
                                          
 Mcnemar's Test P-Value : 4.003e-10       
                                          
            Sensitivity : 0.5628          
            Specificity : 0.9160          
         Pos Pred Value : 0.6974          
         Neg Pred Value : 0.8590          
             Prevalence : 0.2559          
         Detection Rate : 0.1440          
   Detection Prevalence : 0.2065          
      Balanced Accuracy : 0.7394          
                                          
       'Positive' Class : 1               
                        

## Methods: Plan

### Explanation of Code
To quickly summarize, in our experiment, we first used variable selection techniques such as stepwise selection to first obtain the variables which have the highest probability of giving us higher accuracy. Then, after determining the best possible selection of variables, we fit the model on our test set and interpret our results using a confusion matrix to determine the amount of times we receive true positives, true negatives, and times where we get the incorrect answers. 

### Why Stepwise Selection?
In our model, we decided to use a stepwise selection model to help us pick out the most relevant variables in our dataset. Stepwise selection simplifies the model-building process by automating the selection of significant variables, which is particularly useful when there are many potential predictors and no clear indication of the most important ones. By combining both forward and backward selection methods, it adds variables that provide a significant contribution to the model and removes those that do not. The end result is a model which contains variables that have a justifiable impact on the response variable and leads us to a model that balances both complexity and performance
Interpretation

The logistic regression model demonstrates a good level of accuracy (82.56%) in predicting adult annual income in North America, indicating its effectiveness in distinguishing between income levels. However, the moderate Kappa value (0.5112) suggests only moderate agreement beyond chance, and the difference between sensitivity (56.28%) and specificity (91.60%) highlights a potential bias towards predicting lower income levels more accurately. These results suggest the model's utility in addressing the research question but also point towards the need for further refinement, possibly by re-evaluating variable selection or employing techniques like ROC curve analysis to adjust decision thresholds for better balance in predictive performance. Incorporating cross-validation could enhance model robustness and generalization, reducing the risk of overfitting and providing a more balanced prediction for both income classes.

### For lasso:
first, we split the data into a 75/25 train/test split to test the final model with the optimal lambda value obtained from cross validation. We also converted the string response into binary values as that is what glmnet expects as the response for logistic regression. Then we performed lasso cross validation to find the optimal and 1 standard error lambda values. Using the 1 standard error lambda value, we test the fitted model on the training dataset. The 1 standard error lambda value was used in order to have a more conservative model in terms of coefficient shrinkage, as lasso tends to overshrink the coefficients. Cross validation using deviance loss as the loss function of choice allows us to estimate the performance of the model on unseen (test) data, without any strong assumptions. Lasso was used as it allows for both regularization (allowing the model to hopefully generalize better to unseen data), as well as performing variable selection via coordinate descent, as opposed to ridge. An alternative would be elastic net, which combines benefits of ridge (when many predictors are significant, and negates some multicollinearity issues) along with lasso (variable selection).

The final model achieves a good level of accuracy (84.69%) in predicting adult annual income in north america, however it also has the same issues as the model obtained from subset selection, with a kappa level of only 0.5508, which indicates it has better agreement beyond chance than the subset selection model, however still only by a small amount. There is also a large difference between sensitivity and specificity (57.26%, 93.56%), indicating that this model is a better classifier for negative values (incomes below 50k). This may be a result of the disproportionate group sizes of positive vs negative values, which is simply a result of reality as the more people within the population earn less than 50k a year as opposed to more. 

In [4]:
#LASSO:
#drop unused factor levels after filtering, model matrix will create dummy variables for non-existent 
adult_data = droplevels(adult_data)
adult_test = droplevels(adult_test)
X <- model.matrix(~., data = adult_data |> dplyr::select(-income)) 
y <- adult_data$income

lasso_cv <- cv.glmnet(x=X,y=y, alpha=1, family="binomial",nfolds=10)

pred <- predict(lasso_cv,newx=model.matrix(~., data = adult_test |> dplyr::select(-income)),s="lambda.1se")

probs <- exp(pred)/(1+exp(pred))

class_pred <- ifelse(probs <= 0.5,0,1)

confusionMatrix(
    data = as.factor(class_pred),
     reference = as.factor(adult_test$income),
     positive = "1"
    )


Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1966  364
         1  142  361
                                          
               Accuracy : 0.8214          
                 95% CI : (0.8068, 0.8353)
    No Information Rate : 0.7441          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4786          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.4979          
            Specificity : 0.9326          
         Pos Pred Value : 0.7177          
         Neg Pred Value : 0.8438          
             Prevalence : 0.2559          
         Detection Rate : 0.1274          
   Detection Prevalence : 0.1776          
      Balanced Accuracy : 0.7153          
                                          
       'Positive' Class : 1               
                        

# Discussion

Based on our previous discussion in the paragraph above, we can conclude that our logistic regression model is effective in predicting whether or not a given individual will earn more than $50,000 annually, or less than or equal to that. 

Based on our results, this is about what we expected. Using various statistical techniques on a dataset which has information that is relevant to the response variable, we expected to obtain a predictive model that yielded a decent to good accuracy. 
 
Based on the predicted accuracy of our two methods the stepwise selection model shows a slightly higher overall accuracy(82.56%) compared to the Lasso regression model(82.14%). The difference is marginal but might be significant depending on the application. The Kappa statistic for the stepwise selection is higher, suggesting better agreement between the predicted and actual values than the Lasso model. Sensitivity, or the true positive rate, is higher in the stepwise selection model. This indicates that the stepwise model is better at correctly identifying individuals with an income above 50K compared to the Lasso model specificity is slightly lower in the stepwise selection model than in the Lasso regression, which suggests the stepwise model is less accurate in identifying individuals with an income below 50K. The balanced accuracy, which takes into account both sensitivity and specificity, is higher in the stepwise selection model, indicating a better overall performance on both income classes.

In the future, an improvement for our model could be to incorporate other techniques such as cross-validation to ensure we pick a good training and validation set before fitting the model onto our test set. 

All in all, we believe our findings were successful as our predictive model was able to accurately make predictions on income levels based on the given explanatory variables. As mentioned above, our findings would be useful in helping predict the income potential of an individual based on their background, and can be applied in a tremendous number of fields such as developing marketing strategies, real estate planning, and many more.

# References

Becker, Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20
Harmon, C., Oosterbeek, H., & Walker, I. (2003). The returns to education: Microeconomics. Journal of Economic Surveys, 17(2), 115–156. https://doi.org/10.1111/1467-6419.00191 
Hegewisch, A., & Hartmann, H. (2014). Occupational segregation and the gender wage gap: A job half done. PsycEXTRA Dataset. https://doi.org/10.1037/e529142014-001